### Indexing
- Clean and extract text
- Segment text into chunks
- Encode these chunks into vectors
- Store vectors in databases

In [ ]:
# Imports
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
import textract

In [ ]:
# Initialize tokenizer and model for encoding text into vectors
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained('distilbert-base-uncased')

def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


In [ ]:
# Extract text from a PDF (Sample source)
text = textract.process("./data/deep-learning.pdf", method="pdfminer").decode()

# Segment the text into chunks
chunks = [text[i:i+500] for i in range(0, len(text), 500)]
print('Sample chunk:')
print(chunks[1])


In [ ]:
# Encoding and indexing
dim = model.config.hidden_size
index = faiss.IndexFlatL2(dim) # Using L2 distance for simplicity

for chunk in chunks:
    vec = encode_text(chunk)
    index.add(vec)

# Save the index
faiss.write_index(index, "./data/store-chatgpt.faiss")

### Handling retrieval
- Encode the user query
- Compute similarity scores between the query vector and document vectors
- Retrieve the top K similar chunks

In [ ]:
# Handling retrieval
def retrieve(query, k=5):
    # Load index from file, this is replaced with a vector database
    index = faiss.read_index('./data/store-chatgpt.faiss')
    query_vec = encode_text(query)
    D, I = index.search(query_vec, k)
    return [chunks[i] for i in I[0]], D[0]

# Example
query = "What is RAG in AI?"
retrieved_chunks, distances = retrieve(query=query)
print("Retrieved chunks:")
print(retrieved_chunks, distances)

### Generation
- Combine the query and retrieved texts into a coherent prompt
- Generate a response using model

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

def generate_response_with_chatgpt(query, retrieved_chunks, api_key):
    prompt = f"Question: {query}\nContext: " + " ".join(retrieved_chunks)

    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(messages=[{ "role":"user", "content": prompt }], model="gpt-3.5-turbo")

    generated_text = response.choices[0].message.content

    # Saving the response
    file_path = './data/chat-gpt-response.txt'
    
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(generated_text)

    return generated_text

# Calling actions
api_key = os.getenv('OPEN_AI_KEY')
query = "Explain the concept of back propagation in neural networks"
retrieved_chunks, _ = retrieve(query)
response = generate_response_with_chatgpt(query, retrieved_chunks=retrieved_chunks, api_key=api_key)

print('Saved response in text file: ./data/chat-gpt-response.txt')